In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the function to fit
def target_function(x):
    return np.sin(2 * np.pi * x) + 0.5 * x

# Generate training and test datasets
np.random.seed(0)
x_train = np.random.uniform(-1, 1, 20000).reshape(-1, 1)
# x_train2 = np.random.uniform(1, 2, 1000).reshape(-1, 1) 
# x_train3 = np.random.uniform(-2, -1, 1000).reshape(-1, 1)
# x_train = np.concatenate((x_train1, x_train2, x_train3), axis=0)
y_train = target_function(x_train)

x_test = np.random.uniform(-1, 1, 5000).reshape(-1, 1)
y_test = target_function(x_test)

# Define a Neural Network with ReLU activation and dynamic hidden layers
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.layers = len(hidden_size)
        self.weights = []
        self.biases = []
        
        # Initialize weights and biases for each hidden layer
        previous_size = input_size
        for size in hidden_size:
            self.weights.append(np.random.randn(previous_size, size) * np.sqrt(1 / previous_size))
            self.biases.append(np.zeros((1, size)))
            previous_size = size
        
        # Initialize weights and biases for output layer
        self.weights.append(np.random.randn(previous_size, output_size) * np.sqrt(1 / previous_size))
        self.biases.append(np.zeros((1, output_size)))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)
    
    def forward(self, x):
        self.activations = []
        self.z_values = []
        
        # Forward pass through hidden layers
        for i in range(self.layers):
            z = np.dot(x, self.weights[i]) + self.biases[i]
            x = self.relu(z)
            self.z_values.append(z)
            self.activations.append(x)
        
        # Forward pass through output layer
        z = np.dot(x, self.weights[-1]) + self.biases[-1]
        self.z_values.append(z)
        return z
    
    def backward(self, x, y, output, learning_rate=0.01):
        m = y.shape[0]
        
        # Calculate gradients for output layer
        dz = output - y
        dW = np.dot(self.activations[-1].T, dz) / m
        db = np.sum(dz, axis=0, keepdims=True) / m

        # Gradient descent update for output layer
        self.weights[-1] -= learning_rate * dW
        self.biases[-1] -= learning_rate * db
        
        # Backpropagation through hidden layers
        for i in range(self.layers - 1, -1, -1):
            dz = np.dot(dz, self.weights[i+1].T) * self.relu_derivative(self.z_values[i])
            dW = np.dot(x.T if i == 0 else self.activations[i-1].T, dz) / m
            db = np.sum(dz, axis=0, keepdims=True) / m

            # Gradient descent update for hidden layers
            self.weights[i] -= learning_rate * dW
            self.biases[i] -= learning_rate * db

    def train(self, x_train, y_train, epochs=500, learning_rate=0.01):
        for epoch in range(epochs):
            output = self.forward(x_train)
            loss = np.mean((output - y_train) ** 2)
            self.backward(x_train, y_train, output, learning_rate)
            
            if epoch % 20 == 0:
                print(f"\rEpoch {epoch}, Loss: {loss:.4f}, Learning Rate: {learning_rate:.4f}", end="")
            # if epoch % 2000 == 0 and epoch != 0:
            #     learning_rate *= 0.8
            if loss < learning_rate * 0.8:
                learning_rate *= 0.8
        print("\nTraining Complete!")
    
    def predict(self, x):
        return self.forward(x)

# Initialize and train the model with dynamic hidden layers
hidden_layers = [20, 10]  # Adjust hidden layer sizes here
model = SimpleNN(input_size=1, hidden_size=hidden_layers, output_size=1)
model.train(x_train, y_train, epochs=10000, learning_rate=0.05)

# Predict on test set
y_pred = model.predict(x_test)

# Plot results
plt.figure(figsize=(8, 5))
plt.scatter(x_test, y_test, label='True Function', color='blue', s=10)
plt.scatter(x_test, y_pred, label='NN Predictions', color='red', s=10)
plt.legend()
plt.show()


In [ ]:
y_pred = model.predict(x_train)
plt.figure(figsize=(8, 5))
plt.scatter(x_train, y_train, label='True Function', color='blue', s=10)
plt.scatter(x_train, y_pred, label='NN Predictions', color='red', s=10)
plt.legend()
plt.show()